MOUNTING GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


IMPORTING LIBRARIES

In [ ]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # helps find the threshold for grayscale to binary conversion of images
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras

DEIFINING PATHS TO REAL & FORGED DATASET

In [ ]:
genuine_image_paths = "drive/MyDrive/ML MINI PROJECT/real"
forged_image_paths = "drive/MyDrive/ML MINI PROJECT/forged"

PREPROCESSING

1. Converts RGB to Gray Scale

In [ ]:
def rgbgrey(img):
    greyimg = np.zeros((img.shape[0], img.shape[1]))
    for row in range(len(img)):
        for col in range(len(img[row])):
            greyimg[row][col] = np.average(img[row][col])
    return greyimg

2. Converts  Gray Scale to Binary

In [ ]:
def greybin(img):
  blur_radius = 0.8
  img = ndimage.gaussian_filter(img, blur_radius)  
  thres = threshold_otsu(img)
  binimg = img > thres
  binimg = np.logical_not(binimg)
  return binimg

3. Preprocessing Function

In [ ]:
def preproc(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)
    if display:
        plt.imshow(img)
        plt.show()
    grey = rgbgrey(img) #coverting to gray scale
    if display:
        plt.imshow(grey, cmap = matplotlib.cm.Greys_r)
        plt.show()
    binimg = greybin(grey) #coverting to binary
    if display:
        plt.imshow(binimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    r, c = np.where(binimg==1)
    # Now we will make a bounding box with the boundary as the position of pixels on extreme.
    # Thus we will get a cropped image with only the signature part.
    signimg = binimg[r.min(): r.max(), c.min(): c.max()]
    if display:
        plt.imshow(signimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    return signimg

FEATURE EXTRACTION

In [ ]:
def Ratio(img):
    a = 0
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                a = a+1
    total = img.shape[0] * img.shape[1]
    return a/total

In [ ]:
def Centroid(img):
    numOfWhites = 0
    a = np.array([0,0])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                b = np.array([row,col])
                a = np.add(a,b)
                numOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/numOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]

In [ ]:
def EccentricitySolidity(img):
    r = regionprops(img.astype("int8"))
    return r[0].eccentricity, r[0].solidity

In [ ]:
def SkewKurtosis(img):
    h,w = img.shape
    x = range(w)  # cols value
    y = range(h)  # rows value
    #calculate projections along the x and y axes
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    #centroid
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    #standard deviation
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    
    #skewness
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)

    #Kurtosis
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    # 3 is subtracted to calculate relative to the normal distribution
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)

In [ ]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = preproc(path, display=display)
    ratio = Ratio(img)
    centroid = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewness, kurtosis = SkewKurtosis(img)
    retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    return retVal

In [ ]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    temp = getFeatures(path, display=display)
    features = (temp[0], temp[1][0], temp[1][1], temp[2], temp[3], temp[4][0], temp[4][1], temp[5][0], temp[5][1])
    return features

SAVING THE EXTRACTED FEATURES

In [ ]:
def makeCSV():
    if not(os.path.exists('drive/MyDrive/ML MINI PROJECT/FEATURES')):
        os.mkdir('drive/MyDrive/ML MINI PROJECT/FEATURES')
        print('New folder "FEATURES" created')
    if not(os.path.exists('drive/MyDrive/ML MINI PROJECT/FEATURES/Training')):
        os.mkdir('drive/MyDrive/ML MINI PROJECT/FEATURES/Training')
        print('New folder "FEATURES/Training" created')
    if not(os.path.exists('drive/MyDrive/ML MINI PROJECT/FEATURES/Testing')):
        os.mkdir('drive/MyDrive/ML MINI PROJECT/FEATURES/Testing')
        print('New folder "FEATURES/Testing" created')
     
    gpath = genuine_image_paths
     
    fpath = forged_image_paths
    
    for person in range(1,13):
        per = ('00'+str(person))[-3:]
        print('Saving extracted features of Person-',per)
        
        with open('drive/MyDrive/ML MINI PROJECT/FEATURES/Training/training_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Training set
            for i in range(0,3):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(0,3):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')
        
        with open('drive/MyDrive/ML MINI PROJECT/FEATURES/Testing/testing_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Testing set
            for i in range(3, 5):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(3,5):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')

In [ ]:
makeCSV()

Saving extracted features of Person- 001
Saving extracted features of Person- 002
Saving extracted features of Person- 003
Saving extracted features of Person- 004
Saving extracted features of Person- 005
Saving extracted features of Person- 006
Saving extracted features of Person- 007
Saving extracted features of Person- 008
Saving extracted features of Person- 009
Saving extracted features of Person- 010
Saving extracted features of Person- 011
Saving extracted features of Person- 012


MODEL

In [ ]:
from keras import utils as np_utils

In [ ]:
def testing(path):
    feature = getCSVFeatures(path)
    if not(os.path.exists('drive/MyDrive/ML MINI PROJECT/TestFeatures')):
        os.mkdir('drive/MyDrive/ML MINI PROJECT/TestFeatures')
    with open('drive/MyDrive/ML MINI PROJECT/TestFeatures/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, feature))+'\n')

In [ ]:
n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = 'drive/MyDrive/ML MINI PROJECT/FEATURES/Training/training_'+train_person_id+'.csv'
testing(test_image_path)
test_path = 'drive/MyDrive/ML MINI PROJECT/TestFeatures/testcsv.csv'

Enter person's id : 001
Enter path of signature image : drive/MyDrive/ML MINI PROJECT/real/001001_001.png


In [ ]:
def readCSV(train_path, test_path, type2=False):
    # Reading train data
    df = pd.read_csv(train_path, usecols=range(n_input))
    train_input = np.array(df.values)
    train_input = train_input.astype(np.float32, copy=False)  # Converting input to float_32
    df = pd.read_csv(train_path, usecols=(n_input,))
    temp = [elem[0] for elem in df.values]
    correct = np.array(temp)
    corr_train = keras.utils.np_utils.to_categorical(correct,2)      # Converting to one hot
    # Reading test data
    df = pd.read_csv(test_path, usecols=range(n_input))
    test_input = np.array(df.values)
    test_input = test_input.astype(np.float32, copy=False)
    if not(type2):
        df = pd.read_csv(test_path, usecols=(n_input,))
        temp = [elem[0] for elem in df.values]
        correct = np.array(temp)
        corr_test = keras.utils.np_utils.to_categorical(correct,2)      # Converting to one hot
    if not(type2):
        return train_input, corr_train, test_input, corr_test
    else:
        return train_input, corr_train, test_input

In [ ]:
tf.compat.v1.disable_eager_execution()

DEFINING IMPORTANT PARAMETERS FOR THE MLP MODEL

In [ ]:
learning_rate = 0.1
training_epochs = 100
cost_history = np.empty(shape=[1], dtype=float)
n_dim = 9
n_class = 2 #two possible output
print(n_dim)

#define the number of hidden layers and number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

x = tf.compat.v1.placeholder(tf.float32, [None, n_dim])
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.compat.v1.placeholder(tf.float32, [None, n_class])

9


MLP MODEL  (sigmoid activation function)

In [ ]:
def multilayer_perceptron(x, weights, biases):
  # Input layer 
  layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  layer_1 = tf.nn.sigmoid(layer_1)
  # Hidden layer 
  layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
  layer_2 = tf.nn.sigmoid(layer_2)
  # Hidden layer 
  layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
  layer_3 = tf.nn.sigmoid(layer_3)
  # Hidden layer 
  layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
  layer_4 = tf.nn.sigmoid(layer_4)
  # Output layer 
  out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
  return out_layer

Weights and Biases 

In [ ]:
weights = {
  'h1': tf.Variable(tf.compat.v1.truncated_normal([n_dim, n_hidden_1])),
  'h2': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_1, n_hidden_2])),
  'h3': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_2, n_hidden_3])),
  'h4': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_3, n_hidden_4])),
  'out': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_4, n_class]))
}
biases = {
  'b1': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_1])),
  'b2': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_2])),
  'b3': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_3])),
  'b4': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_4])),
  'out': tf.Variable(tf.compat.v1.truncated_normal([n_class]))
} 

In [ ]:
#initialize 
init = tf.compat.v1.global_variables_initializer()

y = multilayer_perceptron(x, weights, biases)
#Cost function 
cost_function = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=y_))
training_step = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
sess = tf.compat.v1.Session()
sess.run(init)

In [ ]:
# Define loss and Optimizer
loss_op = tf.reduce_mean(tf.compat.v1.squared_difference(y, y_))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# For accuracies
pred = tf.nn.softmax(y)  # Apply softmax to logits
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Initializing the variables
init = tf.compat.v1.global_variables_initializer()

In [ ]:
def evaluate(train_path, test_path, type2=False):   
    if not(type2):
        train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path)
    else:
        train_input, corr_train, test_input = readCSV(train_path, test_path, type2)
    ans = 'Random'
    with tf.compat.v1.Session() as sess:
        sess.run(init)
        # Training cycle
        for epoch in range(training_epochs):
            # Run optimization op (backprop) and cost op (to get loss value)
            _, cost = sess.run([train_op, loss_op], feed_dict={x: train_input, y_: corr_train})
            if cost<0.0001:
                break      
        # Finding accuracies
        accuracy1 =  accuracy.eval({x: train_input, y_: corr_train})
        print("Accuracy for training model:", accuracy1)
        prediction = pred.eval({x: test_input})
        
        if prediction[0][1]>prediction[0][0]:
            print('Genuine Image')
            return True
        else:
            print('Forged Image')
            return False

In [ ]:
evaluate(train_path, test_path, type2=True)

Accuracy for training model: 0.5
Genuine Image


True

Sample Paths to Test

In [ ]:
#real: drive/MyDrive/ML MINI PROJECT/real/001001_001.png
#forged: drive/MyDrive/ML MINI PROJECT/forged/021001_001.png